## LSTM!!!

In [1]:
#from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adadelta, Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
text = open('datasetV2twplustag').read().lower()#open('BigData').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

('corpus length:', 896291)
('total chars:', 40)


In [3]:
with open('datasetV2twplustag') as f: #open('BigData') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
f = [x.strip().lower() for x in content] 

end = []
vri = []
for i in range(len(content)):
    end.append(content[i].index('['))
    vri.append(content[i].index(']')-content[i].index('[')+1)
    content[i] = content[i][:-1].lower()
    
sentences = content

In [4]:
maxlen = len(max(content, key=len))

X = np.zeros((sum(vri), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((sum(vri), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    k = sum(vri[:i])-i
    for j in range(1, vri[i]):
        for t, char in enumerate(sentence[:-j]):
            X[k+j-1, t+(maxlen-len(sentence[:-j])), char_indices[char]] = 1
        y[k+j-1, char_indices[sentence[-j]]] = 1

print X.shape

(128049, 144, 40)


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float32')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    choices = range(len(preds))##new
    
    #probas = np.random.multinomial(1, preds, 1)
    
    return np.random.choice(choices, p=preds) #np.argmax(probas)

## Model quality

In [6]:
model = keras.models.load_model('modelKsmall155')

In [7]:
tn = 2
for diversity in [0.2, 0.5, 1.0, 1.3, 1.5]:
    print('diversity:', diversity)

    generated = ''
    sentence = content[tn][:end[tn]+1]#text[start_index: start_index + maxlen]
    generated += sentence
    generated += '['
    #print 'Generating with seed:\n"' + sentence[:] +'"'
    print 'real hashtag: ' + content[tn][end[tn]:]
    #print 'generated hashtag: ['
    #sys.stdout.write(generated+'\n[')
    sys.stdout.write('generated hashtag: [')
    
    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if (next_char==']'):
            break
        
    print '\n'

('diversity:', 0.2)
real hashtag: [mufc]
generated hashtag: []

('diversity:', 0.5)
real hashtag: [mufc]
generated hashtag: [sr]

('diversity:', 1.0)
real hashtag: [mufc]
generated hashtag: [in]

('diversity:', 1.3)
real hashtag: [mufc]
generated hashtag: [fo2]

('diversity:', 1.5)
real hashtag: [mufc]
generated hashtag: [f92]



In [8]:
from difflib import SequenceMatcher

def similar(a,b):
    return SequenceMatcher(None, a, b).ratio()

In [23]:
S = np.zeros((len(sentences), 3))

for i, sentence in enumerate(sentences):
    #if i%10==0:
    #    print i, 
    
    realht = content[i][end[i]:]
    
    for j, diversity in enumerate([1.0, 1.3, 1.5]):
        
        sentence = content[i][:end[i]+1]
        genword = '['
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            genword += next_char
            sentence = sentence[1:] + next_char

            if (next_char==']'):
                break
                
        S[i,j] = similar(genword, realht)
        print "\t real", realht, "\t generated:", genword
    
    

	 real [coyr] 	 generated: [vlhan]
	 real [coyr] 	 generated: [sbpuit]
	 real [coyr] 	 generated: []
	 real [mufc] 	 generated: [ordhnry]
	 real [mufc] 	 generated: [van2w]
	 real [mufc] 	 generated: [eaenm]
	 real [mufc] 	 generated: [rweow]
	 real [mufc] 	 generated: [bxmfymfo1ynnepacboxrip1yevaattpbgenns2nurmsiuiicltmwwdk]
	 real [mufc] 	 generated: [e2hh]
	 real [master] 	 generated: []
	 real [master] 	 generated: [dvt2lglpzrmeyprn2sa]
	 real [master] 	 generated: [s]
	 real [fcb] 	 generated: [wtctn]
	 real [fcb] 	 generated: [ociflobytqf11rtl]
	 real [fcb] 	 generated: [usttnpd]
	 real [chimeforchange] 	 generated: []
	 real [chimeforchange] 	 generated: [epysiiidizdcpesv]
	 real [chimeforchange] 	 generated: [7]
	 real [dataviz] 	 generated: [o]
	 real [dataviz] 	 generated: [l]
	 real [dataviz] 	 generated: [koc]
	 real [vr] 	 generated: [dop]
	 real [vr] 	 generated: [sso]
	 real [vr] 	 generated: [ebfgazlmlt0t5yyly]
	 real [worldmalariaday] 	 generated: [s]
	 real [worldmala

KeyboardInterrupt: 

In [22]:
print np.max(S)
print S.shape

0.5
(10000, 3)


In [15]:
np.save('similarityM', S)

In [28]:
tn = 6
for diversity in [0.2, 0.5, 1.0, 1.3, 1.5]:
    print('diversity:', diversity)

    generated = ''
    sentence = content[tn][:end[tn]+1]#text[start_index: start_index + maxlen]
    generated += sentence
    generated += '['
    #print 'Generating with seed:\n"' + sentence[:] +'"'
    print 'real hashtag: ' + content[tn][end[tn]:]
    #print 'generated hashtag: ['
    #sys.stdout.write(generated+'\n[')
    sys.stdout.write('generated hashtag: [')
    
    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if (next_char==']'):
            break
        
    print '\n'

('diversity:', 0.2)
real hashtag: [dataviz]
generated hashtag: []

('diversity:', 0.5)
real hashtag: [dataviz]
generated hashtag: []

('diversity:', 1.0)
real hashtag: [dataviz]
generated hashtag: []

('diversity:', 1.3)
real hashtag: [dataviz]
generated hashtag: [arfxe]

('diversity:', 1.5)
real hashtag: [dataviz]
generated hashtag: [nsfx5zsosdvpg]

